In [9]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
import pandas as pd

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback

from Furhat_Asks import furhat_questions
from Furhat_Asks import user_model_one
from Furhat_Asks import user_model_two
from Furhat_Asks import user_model_three
from Furhat_Asks import user_model_four
from Furhat_Asks import user_model_five
from Furhat_Asks import user_model_six

import wandb
from wandb.integration.sb3 import WandbCallback

import matplotlib.pyplot as plt



In [10]:
Box(np.array([-6, 0, 1]),np.array([50, 2, 4]),dtype=int).sample()  

array([17,  2,  3])

In [11]:
MultiDiscrete([3,3]).sample()

array([1, 0], dtype=int64)

In [12]:
class SequenceLearningEnv(Env):
    def __init__(self):
        # Actions we can take, reduce, stay, increase valence/task difficulty
        self.action_space = MultiDiscrete([3,3])
    
        # Set start score
        self.score = 0
        
        # Set length counter 6
        self.questions_length = 0
        
        self.task_difficulty = 1  #start with lowest difficulty
        
        self.valence = 1  #start with neutral valence
        
        self.consecutive_bonus = 0
              
        # observation array previous score/current valence/current task difficulty
        self.observation_space = Box(np.array([-6, 0, 0]),np.array([50, 2, 3]),dtype=int)
        
        self.npdata = npdata = np.array([['Question_number','Valence','Task Difficulty','Agent Reward','Participant Score']])
        

        
    def step(self, action):
        #increase questions length by 1
        self.questions_length += 1 
        
        valence_action = action[0] 
        
        task_difficulty_action = action[1]
        
        #valence action
       
        if valence_action == 2:
            if self.valence <= 1:
                self.valence+=1
        elif valence_action ==1:
            pass
        else:
            if self.valence >= 1:
                self.valence-=1
                
        #task difficulty action
        
        if task_difficulty_action == 2:
            if self.task_difficulty <= 3:
                self.task_difficulty+=1
        elif task_difficulty_action == 1:
            pass  
        else:
            if self.task_difficulty >= 2:
                self.task_difficulty-=1
        
        current_sequence, answer = furhat_questions(self.questions_length, self.valence, self.task_difficulty)

        #current_sequence, answer = user_model_two(self.questions_length, self.valence, self.task_difficulty)
        #current_sequence, answer = user_model_three(self.questions_length, self.valence, self.task_difficulty)
        #current_sequence, answer = user_model_four(self.questions_length, self.valence, self.task_difficulty)
        #current_sequence, answer = user_model_five(self.questions_length, self.valence, self.task_difficulty)

        #current_sequence, answer = user_model_six(self.questions_length, self.valence, self.task_difficulty)
        
        # Calculate reward and score
        previous_score=0
        if current_sequence == answer: 
            self.consecutive_bonus += 1
            reward = 5 * self.task_difficulty * self.consecutive_bonus
            self.score += 5 * self.task_difficulty
            previous_score = 5 * self.task_difficulty
        else:            #negative consecutive bonus
            self.consecutive_bonus = 0
            reward = -1
            self.score -= 5
            previous_score = -5
            
            
        
        # Check if sequence quiz has finished
        if self.questions_length >= 6: 
            done = True
        else:
            done = False
        
       
        # Set placeholder for info
        info = {}
        
        # Return step information
        
        self.npdata=np.append(self.npdata, [[self.questions_length, self.valence, self.task_difficulty, reward, self.score]], axis=0)
        
        return np.array([previous_score,self.valence,self.task_difficulty]), reward, done, info

    def render(self):
        # Implement visuals
        pass
    
    def reset(self):
        # Set start score
        self.score = 0
        
        # Set length counter 6
        self.questions_length = 0
        
        self.task_difficulty = 1  #start with lowest difficulty
        
        self.valence = 1  #start with neutral valence
        
        self.consecutive_bonus = 0
        
        
        return np.array([0,self.valence,self.task_difficulty])

In [13]:
env=SequenceLearningEnv()

In [14]:
env.observation_space.sample()
env.reset()

array([0, 1, 1])

In [15]:
#from stable_baselines3.common.env_checker import check_env
#check_env(env, warn=True)

In [16]:
episodes = 3
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

WIP
SKT


AttributeError: 'NoneType' object has no attribute 'replace'

In [ ]:
pdfpdf= pd.DataFrame(env.npdata)

In [ ]:
pdfpdf

In [ ]:
env.action_space.sample()

In [ ]:
env.observation_space.sample()

In [ ]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model_real_learning')
log_path = os.path.join('Training', 'Logs')

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1)

In [ ]:
model.learn(total_timesteps=10)

In [ ]:
pdfpdf= pd.DataFrame(env.npdata)

In [ ]:
pdfpdf

In [ ]:
pdfpdf.to_excel("real_time2.xlsx")

In [ ]:
model.save(PPO_path)

In [ ]:
del model

In [ ]:
model = PPO.load(PPO_path, env=env)

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

In [ ]:
env.close()

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_10')

In [ ]:
training_log_path

In [ ]:
!tensorboard --logdir={training_log_path}

In [ ]:
#%tensorboard --logdir logs/fit

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
evaluate_policy(model, env, n_eval_episodes=3, render=False)

In [ ]:
env.close()